#                               - Computational Neuroscience 2021-2022 Final Project -                    

The functional architecture of the object vision pathway in the human brain was
investigated using functional magnetic resonance imaging to measure patterns
of response in ventral temporal cortex while subjects viewed faces, cats, Þve
categories of man-made objects, and nonsense pictures. A distinct pattern of
response was found for each stimulus category. The distinctiveness of the
response to a given category was not due simply to the regions that responded
maximally to that category, because the category being viewed also could be
identiÞed on the basis of the pattern of response when those regions were
excluded from the analysis. Patterns of response that discriminated among all
categories were found even within cortical regions that responded maximally
to only one category. These results indicate that the representations of faces
and objects in ventral temporal cortex are widely distributed and overlapping.


Filename       : CompNeuro_2021-2022_Final_Project.ipynb

Authors        : Arman Vural Budunoğlu and Can Kocagil

Institution    : Bilkent University Departman of Electric & Electronical Enginering

Class          : EEE482/582 - Computational Neuroscience 

Project Goal   : Implement multi-voxel pattern analyses methods (based on some type of classifier) to
                 decode the category of visual stimuli viewed by a human subject based on their recorded brain activity
                 
Dataset Link   : https://openfmri.org/dataset/ds000105.

Related Papers : Distributed and overlapping representations of faces and objects in ventral temporal cortex

Abstract:

    The functional architecture of the object vision pathway in the human brain was
    investigated using functional magnetic resonance imaging to measure patterns
    of response in ventral temporal cortex while subjects viewed faces, cats, Þve
    categories of man-made objects, and nonsense pictures. A distinct pattern of
    response was found for each stimulus category. The distinctiveness of the
    response to a given category was not due simply to the regions that responded
    maximally to that category, because the category being viewed also could be
    identiÞed on the basis of the pattern of response when those regions were
    excluded from the analysis. Patterns of response that discriminated among all
    categories were found even within cortical regions that responded maximally
    to only one category. These results indicate that the representations of faces
    and objects in ventral temporal cortex are widely distributed and overlapping.

Pipeline:
    
    1) Necessary Installations (If necessary)
    2) Imports
    3) Visual Stimuli and Category Loading
    4) Visual Stimuli Transformations
    5) Explanatory Visual Stimuli Analysis
        
        * PCA
        * T-Stochastic Neighboor Embedding (t-SNE)
        * Linear Discriminate Analysis
        * Uniform Manifold Approximation and Projection (UMAP)
        * Independent Component Analysis (ICA)
        * Non-Negative Matrix Factorization
        * Masking
        
    6) Visual Stimuli Similarity Analysis
    
        * Euclidean Similarity
        * Cosine Similarity
        * Pearson Correlation             
        
    7) Classical ML Algorithms:
    
        * LinearSVC
        * SGDClassifier
        * MLPClassifier
        * Perceptron
        * LogisticRegression
        * LogisticRegressionCV
        * SVC
        * CalibratedClassifierCV
        * PassiveAggressiveClassifier
        * LabelPropagation
        * LabelSpreading
        * RandomForestClassifier
        * GradientBoostingClassifier
        * QuadraticDiscriminantAnalysis
        * RidgeClassifierCV
        * RidgeClassifier
        * AdaBoostClassifier
        * ExtraTreesClassifier
        * KNeighborsClassifier
        * BaggingClassifier
        * BernoulliNB
        * LinearDiscriminantAnalysis
        * GaussianNB
        * NuSVC
        * DecisionTreeClassifier
        * NearestCentroid
        * ExtraTreeClassifier
        * CheckingClassifier
        * DummyClassifier
        
    7) Reported Metrics
        * Accuracy
        * Balanced Accuracy
        * ROC AUC
        * F1-Score
        * Time Taken
        
    8) Deep Learning Algorithms
        * 3-D Convolutional Neural Networks
        * Visual Transformers
        * ...
        
    9) Results Interpretation
    





# Necessary Installations (If necessary)

In [31]:
!pip install umap
!pip install pipreqs
!pip install lazypredict
!pip install nibabel
!pip install nilearn

try:
    import sklearn
    print('Scikit-learn is available, version', sklearn.__version__)
    
except:
    !pip install scikit-learn
    
 
try:
    import cv2
    print('Open-CV is available, version', cv2.__version__)
    
except:
     !pip install opencv-python
    
   
try:
    import seaborn
    print('Seaborn is available, version', seaborn.__version__)
    
except:
     !pip install seaborn


Scikit-learn is available, version 0.23.1
Open-CV is available, version 4.5.1
Seaborn is available, version 0.11.0


# Imports

In [33]:
from __future__ import print_function, division

# Basics:
import numpy as np,pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import os, random, time, sys, cv2, copy, math


# interactive mode
plt.ion()

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

# Dimension Reduction Algorithms:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import FastICA
from sklearn.decomposition import NMF
import umap

# Transformations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Metrics:
from sklearn.metrics import classification_report

# Train-Test Splitter:
from sklearn.model_selection import train_test_split

# For Classical ML algorithms:
from lazypredict.Supervised import LazyClassifier

# Utilies:
from tqdm import tqdm
import pickle

# For distance measurements:
from scipy.spatial.distance import cdist

# Extras:
from abc import abstractmethod
from typing import Callable, Iterable, List

# Set true for Google Colab:
COLAB = False

if COLAB:
    # To access Google Drive:
    from google.colab import drive
    drive.mount("/content/gdrive")

    
# For neuroimaging:
from nibabel.testing import data_path
from nilearn import plotting as nplt
from nilearn.input_data import NiftiMasker
    


print("NumPy Version: ", np.__version__)


root_dir = os.getcwd()
dataset_dir = os.path.join(root_dir,'dataset')

print('Working Directory: \n ', root_dir)


# Creating requirements.txt file
!pip3 freeze > requirements.txt  

NumPy Version:  1.19.1
Working Directory: 
  C:\Users\Administrator\Desktop\VOR


# Utilities

In [26]:
def confusion_matrix(labels:Iterable[list or np.ndarray],
                     preds:Iterable[list or np.ndarray]) -> pd.DataFrame:
    """
        Takes desireds/labels and softmax predictions,
        return a confusion matrix.
        
    """
    label = pd.Series(labels,name='Actual')
    pred = pd.Series(preds,name='Predicted')
    return pd.crosstab(label,pred)

def visualize_confusion_matrix(data:np.ndarray,
                               normalize:bool = True,
                               title:str = " ") -> None:
    
    if normalize:

        data /= np.sum(data)

    plt.figure(figsize=(15,15))
    sns.heatmap(data, 
                fmt='.2%',
                cmap = 'Greens')

    plt.title(title)
    plt.show()



def accuracy(labels,preds):
      return (np.sum(preds == labels) / labels.shape) * 100


def save_obj(obj:object, path:str = None) -> None:
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

        
def load_obj(path:str = None) -> object:
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


def save(data:np.ndarray = None,path:str = None) -> None:
    np.save(path + '.npy', data)


def load(path:str = None) -> np.ndarray:
    return np.load(path + '.npy')    
    
    
def random_seed(Func:Callable):
    def _random_seed(*args, **kwargs):
        np.random.seed(42)
        random.seed(42)
        result = Func(*args, **kwargs)
        return result
    return _random_seed


@random_seed 
def timeit(Func:Callable):
    def _timeStamp(*args, **kwargs):
        since = time.time()
        result = Func(*args, **kwargs)
        time_elapsed = time.time() - since

        if time_elapsed > 60:
           print('Time Consumed : {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  
        else:        
          print('Time Consumed : ' , round((time_elapsed),4) , 's')
        return result
    return _timeStamp

@random_seed 
@timeit
def plot_2d(component1:np.ndarray, component2:np.ndarray, y = None) -> None:
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=y, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        )
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000,height=1200)                 
    fig.layout.template = 'plotly_dark'
    
    fig.show()
 
@random_seed 
@timeit
def plot_3d(component1: np.ndarray,component2 : np.ndarray,component3 :np.ndarray, y = None) -> None:
    fig = go.Figure(data=[go.Scatter3d(
            x=component1,
            y=component2,
            z=component3,
            mode='markers',
            marker=dict(
                size=10,
                color=y,                # set color to an array/list of desired values
                colorscale='Rainbow',   # choose a colorscale
                opacity=1,
                line_width=1
            )
        )])
    # tight layout
    fig.update_layout(margin=dict(l=50,r=50,b=50,t=50),width=1800,height=1000)
    fig.layout.template = 'plotly_dark'

    fig.show()

# Visual Stimuli and Category Loading

In [29]:
%%time

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


#  Visual Stimuli Transformations

In [28]:
%%time
## Standardizing the data
# x = StandardScaler().fit_transform(x)

# Normalizing data:
# MinMaxScaler().fit_transform(x)


Wall time: 0 ns


# Explanatory Visual Stimuli Analysis

In [ ]:
%%time

## PCA

In [ ]:
%%time

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principal = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

#plot_2d(principalComponents[:, 0],principalComponents[:, 1])

## T-Stochastic Neighboor Embedding (t-SNE)

In [ ]:
%%time

pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(x)
tsne = TSNE(random_state = 42, n_components=3,verbose=0, perplexity=40, n_iter=400).fit_transform(pca_result_50)

## Linear Discriminate Analysis

In [ ]:
%%time

X_LDA = LDA(n_components=3).fit_transform(standardized_data,y)

## Uniform Manifold Approximation and Projection (UMAP)

In [ ]:
%%time


reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(x)

## Independent Component Analysis (ICA)

In [ ]:
%%time

fast_ica = FastICA(n_components = 3)

## Non-Negative Matrix Factorization

In [ ]:
%%time

nmf = NMF(n_components = 3, max_iter=500)

## Masking

In [ ]:
%%time



# Classical ML Algorithms

In [ ]:
%%time
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models)

# Deep Learning Algorithms

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image


# PyTorch's versions:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

# We will be working with GPU:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device : ' , device)

# Number of GPUs available. 
num_GPU = torch.cuda.device_count()
print('Number of GPU : ', num_GPU)

config = {'batch_size': 4}

transform = transforms.Compose([
                                transforms.ColorJitter([0.9,0.9]),
                                transforms.RandomGrayscale(p = 0.3),
                                transforms.RandomAffine((-30,30)),
                                transforms.RandomPerspective(),
                                transforms.GaussianBlur(3),
                                transforms.RandomHorizontalFlip(p = 0.2),
                                transforms.RandomVerticalFlip(p = 0.2),

                                # Important parts, above can be ignored
                                transforms.Resize((224,224))),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = (0.5,0.5,0.5),
                                                     std  = (0.5,0.5,0.5))       
                                
]))

dataloader = torch.utils.data.DataLoader(dataset = CelebrityData,
                                         shuffle = True,
                                         batch_size = config['batch_size'],
                                         num_workers = num_GPU * 4,
                                         drop_last = True,
                                         pin_memory = True)


@torch.no_grad()
def predict(inputs):
    
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)  
      return preds

## 3-D Convolutional Neural Network

In [34]:
%%time


class Model(nn.Module):
    def __init__(self,model = None):
        super(Model,self).__init__()
        if model is not None:
            self.model = model    
        else:
            self.model = nn.Sequential(
            self.conv_block(3,32,0.1),
            self.conv_block(32,64,0.15),
            self.conv_block(64,128,0.2),
            self.conv_block(128,256,0.25),
            self.conv_block(256,512,0.3),
            self.conv_block(512,1024,0.35),
            nn.Flatten(),
            self.linear_block(1024,512,0.4),
            self.linear_block(512,256,0.4),
            nn.Linear(256,136)
            )     

    def forward(self,img):    
        return self.model(img)  

    @staticmethod
    def conv_block(in_channel,out_channel,p):
        return nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 3),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout2d(p)
            )

    @staticmethod
    def linear_block(in_ftrs,out_ftrs,p):
        return nn.Sequential(
            nn.Linear(in_ftrs,out_ftrs),
            nn.BatchNorm1d(num_features=out_ftrs),
            nn.ReLU(),
            nn.Dropout(p)
            )

net = Model().to(device)


print('Traniable parameter of the model: ' , sum(param.numel() for param in net.parameters() if param.requires_grad == True))

NameError: name 'nn' is not defined

## Visual Transformers 

In [ ]:
%%time

# Results Interpretation

# Environmental

In [ ]:
%env